In [77]:
import numpy as np
import cv2 as cv
from collections import defaultdict, OrderedDict
import json

In [78]:
vid_num = 2
events = defaultdict(dict)
ball_radius = 25
cap = cv.VideoCapture(f'imp/videos/videos_0/{vid_num}.mov')
frame_rate = 5*round(cap.get(cv.CAP_PROP_FPS)/5)
total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
backSub = cv.createBackgroundSubtractorMOG2()

In [79]:
def annotate(frame, frame_i):
    frame_copy = frame.copy()
    label_count = 2
    cv.putText(frame_copy, str(frame_i), (int(frame_width/2) - 50, 100), cv.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 5)
    for key in events[str(frame_i)].keys():
        if key == "ballCoord":
            x, y = events[str(frame_i)][key]
            cv.rectangle(frame_copy, (x-ball_radius, y-ball_radius), (x+ball_radius, y+ball_radius), (255, 0, 0), 1)
        elif key == "shot":
            x_pos = 50 if events[str(frame_i)]["shot"]["player"] == "near" else frame_width - 500
            cv.putText(frame_copy, events[str(frame_i)]["shot"]["name"], (x_pos, 100), cv.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 5)
        else:
            # bounce
            cv.putText(frame_copy, key, (int(frame_width/2) - 100, 200), cv.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 5)
            label_count += 1
    cv.imshow('frame', frame_copy)

In [80]:
def handleMouseClick(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONUP:
        events[str(param["frameNum"])]["ballCoord"] = (x, y)
        annotate(param["frame"], param["frameNum"])

In [81]:
shot_mappings = {
    96: {"name": "F Loop", "player": "near", "key": "`"},
    49: {"name": "B Loop", "player": "near", "key": "1"},
    50: {"name": "F Drive", "player": "near", "key": "2"},
    51: {"name": "B Drive", "player": "near", "key": "3"},
    52: {"name": "F Flick", "player": "near", "key": "4"},
    53: {"name": "B Flick", "player": "near", "key": "5"},
    54: {"name": "F Lob", "player": "near", "key": "6"},
    55: {"name": "B Lob", "player": "near", "key": "7"},
    56: {"name": "F Push", "player": "near", "key": "8"},
    57: {"name": "B Push", "player": "near", "key": "9"},
    48: {"name": "F Chop", "player": "near", "key": "0"},
    45: {"name": "B Chop", "player": "near", "key": "-"},
    61: {"name": "F Smash", "player": "near", "key": "="},
    9: {"name": "B Smash", "player": "near", "key": "TAB"},
    113: {"name": "F Block", "player": "near", "key": "q"},
    119: {"name": "B Block", "player": "near", "key": "w"},
    101: {"name": "F Pend Serve", "player": "near", "key": "e"},
    114: {"name": "B Pend Serve", "player": "near", "key": "r"},
    116: {"name": "Other", "player": "near", "key": "t"},
    121: {"name": "Win Point", "player": "near", "key": "y"},
    
    126: {"name": "F Loop", "player": "far", "key": "~"},
    33: {"name": "B Loop", "player": "far", "key": "!"},
    64: {"name": "F Drive", "player": "far", "key": "@"},
    35: {"name": "B Drive", "player": "far", "key": "#"},
    36: {"name": "F Flick", "player": "far", "key": "$"},
    37: {"name": "B Flick", "player": "far", "key": "%"},
    94: {"name": "F Lob", "player": "far", "key": "^"},
    38: {"name": "B Lob", "player": "far", "key": "&"},
    42: {"name": "F Push", "player": "far", "key": "*"},
    40: {"name": "B Push", "player": "far", "key": "("},
    41: {"name": "F Chop", "player": "far", "key": ")"},
    95: {"name": "B Chop", "player": "far", "key": "_"},
    43: {"name": "F Smash", "player": "far", "key": "+"},
    25: {"name": "B Smash", "player": "far", "key": "SHFT TAB"},
    81: {"name": "F Block", "player": "far", "key": "Q"},
    87: {"name": "B Block", "player": "far", "key": "W"},
    69: {"name": "F Pend Serve", "player": "far", "key": "E"},
    82: {"name": "B Pend Serve", "player": "far", "key": "R"},
    84: {"name": "Other", "player": "far", "key": "T"},
    89: {"name": "Win Point", "player": "far", "key": "Y"},
}

In [82]:
frame_i = 0
is_back_sub = False
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        cap.release()
        cv.destroyAllWindows()
        cv.waitKey(1)
        break
    cv.setMouseCallback('frame', lambda event, x, y, flags, param: handleMouseClick(event, x, y, flags, param={"frame": frame, "frameNum": frame_i}))
    annotate(frame, frame_i)
    back_sub_frame = backSub.apply(frame)
    key = -1
    while key not in [0, 1, 2, 3]:
        cur_frame = frame
        key = cv.waitKey(0)
        if key == 0: ##### ARROWS #####
            # up arrow
            skip_frames = 30
            frame_i += skip_frames
            cap.set(cv.CAP_PROP_POS_MSEC, cap.get(cv.CAP_PROP_POS_MSEC) + skip_frames * 1000 / frame_rate)
        elif key == 1:
            # down arrow
            skip_frames = 5
            frame_i -= skip_frames
            cap.set(cv.CAP_PROP_POS_MSEC, cap.get(cv.CAP_PROP_POS_MSEC) - skip_frames * 1000 / frame_rate)
        elif key == 2:
            # back arrow
            is_back_sub = False
            frame_i = frame_i - 1 if frame_i > 0 else frame_i
            cap.set(cv.CAP_PROP_POS_MSEC, cap.get(cv.CAP_PROP_POS_MSEC)-1000/frame_rate)
        elif key == 3:
            # next arrow
            is_back_sub = False
            frame_i += 1
        elif key in shot_mappings: ##### SHOT TYPES #####
            is_back_sub = False
            events[str(frame_i)]["shot"] = {"name": shot_mappings[key]["name"], "player": shot_mappings[key]["player"]}
            annotate(cur_frame, frame_i)
        elif key == ord('b'):
            # indicate bounce
            events[str(frame_i)]["bounce"] = True
            annotate(cur_frame, frame_i)
        elif key == ord('n'):
            # indicate net contact
            events[str(frame_i)]["net"] = True
            annotate(cur_frame, frame_i)
        elif key == ord('m'):
            if not is_back_sub:
                cur_frame = back_sub_frame
            is_back_sub = not is_back_sub
            annotate(cur_frame, frame_i)
        elif key == 127:
            # DEL clear screen
            events[str(frame_i)] = {}
            annotate(cur_frame, frame_i)
        elif key == 27:
            # ESC to quit
            cap.release()
            cv.destroyAllWindows()
            cv.waitKey(1)
            break


    

In [83]:
# filter out all empty frames
filtered_events = dict()
for (key, value) in events.items():
    if value:
        filtered_events[key] = value
with open(f'imp/videos/labels_0/{vid_num}.json', "w") as outfile:
    json.dump(OrderedDict(sorted(filtered_events.items(), key=lambda t: int(t[0]))), outfile)

: 

In [46]:
import cv2 as cv
import numpy as np
while True:
    cv.imshow("test", np.zeros((100, 100, 3), dtype=np.uint8))
    key = cv.waitKey(0)
    print(key)
    if key == 27:
        cv.destroyAllWindows()
        break

127
127
127
127
27
